# LatentSim!

In [2]:
import numpy as np
from latent_sim import LatentSim

In [3]:
ls = LatentSim()

In [4]:
hub = "/Users/afq/Google Drive/networks/training_water_slgc_logp/"

In [5]:
ls.load_model(hub+"Classifying_2,5,12,24,sigmoid",
             scale_file="data_files/water_iapw_logp_ranges.csv",logp=True)

INFO:tensorflow:Restoring parameters from /Users/afq/Google Drive/networks/training_water_slgc_logp/Classifying_2,5,12,24,sigmoid/final_variables


In [6]:
ls.decode(np.array([[0.0,0.0]]))

array([[3.8475235e+02, 1.4969842e+05, 4.2445288e+02, 1.6894192e+06]],
      dtype=float32)

# Finding initial conditions

We know we only need 2 variables to specify a point on the surface. Most of the time we just want to provide a p and a T. 
However, the econder assumes that the input is on the manifold. 
The user could go through the effort of looking up rho and h, but that's a pain!
We want to let the user specify only 2 numbers and find the intersection on the manifold.

If we just try to plug in the result with "reasonable" fillers for the two unknowns we notice that we encoded onto the wrong point on the manifold:

In [7]:
q = ls.encode(np.array([[250.0, 1.0e3, ls.scale[0,2],ls.scale[0,3]]]))
print q
print ls.decode(q)

[[-0.15995619 -0.00981059]]
[[2.8674936e+02 1.5104628e+03 5.2928625e+02 1.2262239e+06]]


This means:
\begin{equation}
T,p \neq D_{Tp}(E(T,p,?,?))
\end{equation}
Unfortunately this means we have to start solving the equations.
\begin{equation}
R(q) = D_{Tp}(q) - \{T^*,p^*\}
\end{equation}
with a good initial guess of
\begin{equation}
q^0 = E(T^*,p^*,?,?)
\end{equation}

In [8]:
ls.encode(
    np.array([[2.50000000e+02, 1.00000000e+03, 5.41351758e+02 ,1.35349597e+06]]) )

array([[-0.16674453, -0.01538956]], dtype=float32)

We can wrap this process up in a nice interface,

In [9]:
q0 = ls.find_point(T=350,p=5.0e3)

2.0177149662825964 [[ 0.02969093 -0.02830379]]
0.04250428280086618 [[0.05607678 0.4709995 ]]
0.005958300048781399 [[0.02620749 0.4407598 ]]
0.0005218058149165234 [[0.02693096 0.44667402]]
4.922141351954685e-06 [[0.02670286 0.44714332]]
1.0404310085922999e-08 [[0.02670108 0.4471479 ]]
Found point at  [[ 3.5000000e+02  4.9999980e+03 -1.8482666e+00  2.6498230e+06]]  after  5  iterations.


Note that we're working in 32-bit precision, so that's as close as we're going to get!

# Building the equations

In [10]:
ls.build_dae()

# Running a simulation

In [11]:
def schedule(t):
    if t<1000.0:
        ls.set_params(T_inf=800,p_inf=5.0e3)
    elif t<2000.0:
        ls.set_params(T_inf=800,p_inf=3.0e7)
    elif t<3000.0:
        ls.set_params(T_inf=400,p_inf=3.0e7)
    elif t<4000.0:
        ls.set_params(T_inf=400,p_inf=5.0e3)

In [12]:
Dt = 0.05
ls.set_params(Dt=Dt)
ls.set_params(k_p=10.0,k_T=10000.0)
t = 0
t_max = 200.0
q = q0.copy()
qs,ss,ts = [],[],[]
while t<t_max:
    t+=Dt
    schedule(t)
    q = ls.solve_a_time_step(q)
    if np.isnan(q).any() or np.isinf(q).any():
        print "NaN encountered at t=",t
        break
    qs.append(q.copy())
    ss.append(ls.decode(q).copy())
    ts.append(t)

2 1.950733e-07
16 1.7689935e-07
99 8.303364e-07
3 1.8539995e-07
2 1.3492237e-08
99 2.843001e-07
3 6.202092e-08
2 1.5095743e-07
99 1.387282e-06
7 5.7981964e-08
2 1.0802927e-07
99 2.6925682e-07
2 1.6378343e-07
5 5.3081372e-08
99 3.7852652e-07
2 1.6424606e-07
2 1.03492354e-07
99 8.6371347e-07
7 1.5344119e-07
3 4.188019e-08
4 1.6302027e-07
99 1.2075589e-06
3 1.6249047e-07
3 8.036551e-08
99 1.0869718e-06
99 1.2385557e-06
3 1.8603474e-07
99 4.7947884e-07
7 1.7615011e-07
3 4.6754714e-08
99 7.5209005e-07
99 2.8892277e-06
5 1.3869082e-07
20 1.532296e-07
99 0.020243403
99 0.054568943
99 0.054464974
99 0.054493707
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484
99 0.054403484

KeyboardInterrupt: 

In [ ]:
ts = np.array(ts)
ss = np.array(ss)
qs = np.array(qs)

In [ ]:
from matplotlib import pylab as plt
%matplotlib inline
legends=['T','p','rho','h']
for i,name in enumerate(legends):
    plt.figure()
    plt.plot(ts,ss[:,0,i])
    plt.ylabel(name)

In [ ]:
legends=['q0','q1']
for i,name in enumerate(legends):
    plt.figure()
    plt.plot(ts,qs[:,0,i])
    plt.ylabel(name)